In [45]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import torch

# Load your test dataset
test_dataset = load_dataset("md_gender_bias", name="convai2_inferred")["test"]

# Replace 'regression_model' with the actual path where you saved your model
model_path = 'model_output'

# Load the pre-trained model and tokenizer
loaded_model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", max_length=1024)

def process_dataset(dataset):
    new_dataset_dict = {
        'text': [entry['text'] for entry in dataset],
        'labels': [entry['binary_score'] if entry['binary_label'] == 1 else 1 - entry['binary_score'] for entry in dataset]
    }
    new_dataset = Dataset.from_dict(new_dataset_dict)
    return new_dataset

# Tokenize the test dataset
test_dataset = process_dataset(test_dataset)
tokenized_test_dataset = test_dataset.map(lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True), batched=True)



# Set up Trainer with custom data collator
trainer = Trainer(
    model=loaded_model,
)

# Make predictions on the test dataset
predictions = trainer.predict(tokenized_test_dataset)

# Extract the predicted values
predicted_values = predictions.predictions.squeeze().tolist()


In [47]:
# test cell
text = "hello what are doing today ?"
tokenized_input = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
with torch.no_grad():
    logits = model(**tokenized_input).logits

# Convert logits to predicted value
predicted_value = logits.squeeze().item()

# Print the predicted value
print("Predicted Value:", predicted_value)

Predicted Value: 0.5456048250198364
